# <center>CSE2049 Context Aware Computing</center>
## <center>Context Aware Assistant</center>

## Name: Swadeep, Abhinav Nair
## Reg. No.: 19BPS1030, 19BPS1007

In [235]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import re
from location import getLocation,getDistance

In [236]:
_USER_="Swadeep"

In [237]:
def getAltitude(pr):
    return 44330*(1-(pr/1013.25)**(1/5.255))

In [238]:
df=pd.read_csv('Dataset.csv')
df.head()

,Latitude,Longitude,Altitude,Band_Latitude,Band_Longitude,Band_Altitude,Mobile_Alarm,Mobile_Volume,Mobile_Mic,Mobile_Battery,Heart_Rate,SpO2,Temperature,Fitness_Battery,TV_Power,TV_Volume,Timestamp
0,11.98189,78.66505,1013.25,11.98189,78.66505,1013.25,0,5,0,80,80,98,98,80,0,0,12-10-2021 00:00
1,11.98189,78.66505,1013.25,11.98189,78.66505,1013.25,0,5,0,80,80,98,98,80,0,0,12-10-2021 00:05
2,11.98189,78.66505,1013.25,11.98189,78.66505,1013.25,0,5,0,80,80,98,98,80,0,0,12-10-2021 00:10
3,11.98189,78.66505,1013.25,11.98189,78.66505,1013.25,0,5,0,80,80,98,98,80,0,0,12-10-2021 00:15
4,11.98189,78.66505,1013.25,11.98189,78.66505,1013.25,0,5,0,80,80,98,98,80,0,0,12-10-2021 00:20


In [239]:
class User:
    Name=_USER_
    def __init__(self,status):
        self.Status=status

class Mobile:
    def __init__(self,alarm,volume,mic,battery,Lat,Long,Alt):
        self.Alarm=alarm
        self.Volume=volume
        self.Mic=mic
        self.Charge=battery
        self.Latitude=Lat
        self.Longitude=Long
        self.Altitude=Alt

class FitBand:
    def __init__(self,heart,spo2,battery,temperature,Lat,Long,Alt):
        self.Heart_Rate=heart
        self.oxygen=spo2
        self.Charge=battery
        self.Temp=temperature
        self.Latitude=Lat
        self.Longitude=Long
        self.Altitude=Alt

class TV:
    def __init__(self,volume,status):
        self.Volume=volume
        self.Status=status

class Restaurant:
    def __init__(self,name,Lat,Long,Alt,status,avail,hist):
        self.Name=name
        self.Latitude=Lat
        self.Longitude=Long
        self.Altitude=Alt
        self.Status=status
        self.Available=avail
        self.History=hist

class Home:
    #Loc=getLocation()
    #Latitude=Loc[0]
    #Longitude=Loc[1]
    Latitude=11.98189
    Longitude=78.66505
    Altitude=0

In [240]:
House=Home()
print("House latitude, longitude and altitude is:",House.Latitude,House.Longitude)

House latitude, longitude and altitude is: 11.98189 78.66505


In [241]:
resto=pd.read_csv('Restaurant.csv')
resto.head()
Resto=[]
histall={}
for index,row in resto.iterrows():
    alt=getAltitude(row["Altitude"])
    if(isinstance(row["History"], float)):
        items=[]
    else:
        items=row["History"].split(",")
    if(isinstance(row["Available"], float)):
        avail=[]
    else:
        avail=row["Available"].split(",")
    hist={}
    for i in items:
        if (i in hist):
            hist[i] +=1
            histall[i]+=1
        else:
            hist[i]=1
            histall[i]=1

    hist=sorted(hist.items(), key =
             lambda kv:(kv[1], kv[0]),reverse=True)
    Resto.append(Restaurant(row["Name"].upper(),row["Latitude"],row["Longitude"],alt,row["Status"],avail,hist))

histall=sorted(histall.items(), key =
             lambda kv:(kv[1], kv[0]),reverse=True)


# List Of Primary Context

1) User Longitude
2) User Latitude
3) Phone Volume
4) Mic Status
5) Battery Levels
6) TV Status
7) TV Volume
8) Heart Rate
9) SpO2
10) Pressure

# List of Derived Context

1. User Altitude
2. User Status

# Context Aware Behaviours:

1) When its lunchtime/dinnertime, the assistant will automatically find nearby restaurants and give the recommended dishes.

2) When the user is at home and the TV is playing, if a phone call comes, the
assistant will automatically reduce the volume of TV so that he can speak
peacefully. It increases volume after the call ends

3) When the user is sleeping, and the alarm rings and the user is still asleep, the assistant automatically increases the volume of the phone so that the person wakes up.

4) When the user faces any medical difficulty, it automatically diagnoses the user and sends a report to the doctor.

5) If user is outside and TV is on, automatically shut off TV.

6) If the user is awake and alarm starts ringing, we switch off the alarm.


# Conflicting Situation:

1. If person is watching TV and discussing about the TV show on the phone, we need not decrease volume, but it will be a conflict. This can be solved by creating a derived context about show on TV and Matter of discussion on phone. If they are related, we don't have to decrease the TV volume.

2. If person is home and it is Dinner/ Lunch time, the system will suggest restaurants which is not needed as he is at home. This conflict can be solved by checking the location of the user at lunch/ dinner time.

3. If it's dinner/lunch time and the user has already eaten food, our system will still give recommendations. This conflicting situation is very hard to solve as there is no real way to keep track if the user has eaten food or not. One solution to it is checking if he visited any restaurant before lunch or dinnertime.

# Time Behaviour:

It can be seen that our project is completely time dependent. 
1. We increase volume per 5 minutes if user doesn't wake up.
2. We send lunch and dinner recommendation when it is time.
3. We keep updating the primary context values every 5 minutes to simulate real-time working and response.

In [242]:
cols=df.columns
lunch_start="13:20"
lunch_end="14:00"
dinner_start="20:00"
dinner_end="21:00"
ls = datetime.strptime(lunch_start,'%H:%M')
le = datetime.strptime(lunch_end,'%H:%M')
ds = datetime.strptime(dinner_start,'%H:%M')
de = datetime.strptime(dinner_end,'%H:%M')

Alarm_status=0
lunch_alert=0
dinner_alert=0
for index,row in df.iterrows():
    time=row["Timestamp"]
    pt = datetime.strptime(time,'%d-%m-%Y %H:%M')

    alt=getAltitude(row["Altitude"])
    dist=getDistance((float)(row["Band_Latitude"]),(float)(House.Latitude),(float)(row["Band_Longitude"]),(float)(House.Longitude))
    loc="Inside"
    if(dist>800):
        loc="Outside"
    user=User(loc)
    
    mobile=Mobile(row["Mobile_Alarm"],row["Mobile_Volume"],row["Mobile_Mic"],row["Mobile_Battery"],row["Latitude"],row["Longitude"],alt)

    alt=getAltitude(row["Band_Altitude"])
    band=FitBand(row["Heart_Rate"],row["SpO2"],row["Fitness_Battery"],row["Temperature"],row["Band_Latitude"],row["Band_Longitude"],alt)

    tv=TV(row["TV_Volume"],row["TV_Power"])

    if (pt.time()>ls.time() and pt.time()<le.time() and lunch_alert==0):
        lunch_alert=1
        print("\nTime: ",pt)
        print("It's Lunch Time. Available nearby restaurants:")
        for i in range (len(Resto)):
            if(Resto[i].Status==1):
                rdist=getDistance((float)(Resto[i].Latitude),(float)(band.Latitude),(float)(Resto[i].Longitude),(float)(band.Longitude))
                if(rdist<1000):
                    print("")
                    print(Resto[i].Name)
                    print("Recommended Dishes:")
                    for j in range(len(Resto[i].History)):
                        if Resto[i].History[j][0] in Resto[i].Available:
                            print(Resto[i].History[j][0])

    elif (pt.time()>ds.time() and pt.time()<de.time() and dinner_alert==0):
        dinner_alert=1
        print("\nTime: ",pt)
        print("It's Dinner Time. Available nearby restaurants:")
        for i in range (len(Resto)):
            if(Resto[i].Status==1):
                rdist=getDistance((float)(Resto[i].Latitude),(float)(band.Latitude),(float)(Resto[i].Longitude),(float)(band.Longitude))
                if(rdist<1000):
                    print("")
                    print(Resto[i].Name)
                    print("Recommended Dishes:")
                    for j in range(len(Resto[i].History)):
                        if Resto[i].History[j][0] in Resto[i].Available:
                            print(Resto[i].History[j][0])

    if (pt.time()>le.time() and lunch_alert==1):
        lunch_alert=0
    
    if(pt.time()>de.time() and dinner_alert==1):
        dinner_alert=0

    if(user.Status=="Inside"):
        if(tv.Status==1 and mobile.Mic==1 and tv.Volume>0):
            tv.Volume=0
            print("\nTime: ",pt)
            print("User is speaking on phone. We reduce TV Volume.")
        elif(tv.Status==1 and tv.Volume==0 and mobile.Mic==0):
            tv.Volume=5
            print("\nTime: ",pt)
            print("User has finished his call. We increase TV Volume.")
    else:
        if(tv.Status==1):
            tv.Status=0
            print("\nTime: ",pt)
            print("User is outside. So we switch off the TV.")

    if(band.Heart_Rate>100 and band.oxygen<90):
        print("\nTime: ",pt)
        print("ALERT! USER IS FACING HEART ISSUES. CALLING AMBULANCE!")

    elif(band.oxygen<80):
        print("\nTime: ",pt)
        print("ALERT! USER FACING BREATHING ISSUES. CALLING AMBULANCE!")

    if(band.Temp>100):
        print("\nTime: ",pt)
        print("ALERT! USER HAS FEVER. CALLING AMBULANCE!")

    elif(band.Temp<80 and band.Temp>40):
        print("\nTime: ",pt)
        print("ALERT! USER HAS HYPOTHERMIA. CALLING AMBULANCE!")
        
    elif(band.Temp<-480):
        print("\nTime: ",pt)
        print("ALERT! USER HAS A WRONG READING.")

    if(mobile.Alarm==1 and Alarm_status==0):
        Alarm_status=1
        print("\nTime: ",pt)
        print("Alarm started ringing.")
        if(mobile.Latitude!=band.Latitude or mobile.Longitude!=band.Longitude or mobile.Altitude!=band.Altitude):
            print("\nTime: ",pt)
            print("User is awake but away from mobile. We switch off the alarm.")
            Alarm_status=0

    elif(mobile.Alarm==1 and Alarm_status==1):
        if(mobile.Volume<10):
            print("\nTime: ",pt)
            print("User didn't wake up. We increase volume.")
            mobile.Volume=mobile.Volume+1
        else:
            print("\nTime: ",pt)
            print("User didn't wake up. Max volume reached.")

    if(Alarm_status==1 and mobile.Alarm==0):
        print("\nTime: ",pt)
        print("User Woke up.")
        Alarm_status=0
        mobile.Volume=5


Time:  2021-10-12 06:30:00
Alarm started ringing.

Time:  2021-10-12 06:35:00
User didn't wake up. We increase volume.

Time:  2021-10-12 06:40:00
User didn't wake up. We increase volume.

Time:  2021-10-12 06:45:00
User Woke up.

Time:  2021-10-12 09:00:00
User is speaking on phone. We reduce TV Volume.

Time:  2021-10-12 09:25:00
User has finished his call. We increase TV Volume.

Time:  2021-10-12 11:55:00
User is outside. So we switch off the TV.

Time:  2021-10-12 13:25:00
It's Lunch Time. Available nearby restaurants:

BEDROCK
Recommended Dishes:
Burger
Thali
Pizza

BERTHA'S RESTAURANT
Recommended Dishes:
Thali
Pizza

BIRCHES RESTAURANT
Recommended Dishes:
Thali
Pizza

BLARNEY STONE INN
Recommended Dishes:
Thali
Pizza

AMBROSIA'S
Recommended Dishes:
Burger
Thali
Pizza

Time:  2021-10-12 20:04:00
It's Dinner Time. Available nearby restaurants:

BAY CAFE
Recommended Dishes:
Thali
Pizza

BELT STREET PUB
Recommended Dishes:
Thali
Pizza

BELVEDERE TOWERS CAF
Recommended Dishes:
Thal

# Unexpected / new context occurrence handling

1) Hardware Failure: When the volume is greater than 0 and TV status is 0, we can say that the TV hardware has failed.
2) Hardware Failure: If the change in location of sensor in 5 mins is very high we can tell that the hardware has failed.
3) Software Failure: When the user is not wearing the band, the heart rate, temperature wouldn't be measured correctly. We have to add this clause in the code.
4) Software Failure: If any of the geo location sensors fail and gives different value, then the system will assume that the person is awake and alarm won't ring even though he's asleep. : We can solve this by ignoring the fact that user is away and ring alarm.
5) Conflict: When user is home and its eating time, we don't have to show the restaurant recommendations
6) Conflict: If the user is recording a scene on TV using his mic, Mic status will be 1 and the volume will decrease but it shouldn't.

In [243]:
cols=df.columns
lunch_start="13:20"
lunch_end="14:00"
dinner_start="20:00"
dinner_end="21:00"
ls = datetime.strptime(lunch_start,'%H:%M')
le = datetime.strptime(lunch_end,'%H:%M')
ds = datetime.strptime(dinner_start,'%H:%M')
de = datetime.strptime(dinner_end,'%H:%M')
lat=-1
long=-1
Alarm_status=0
lunch_alert=0
dinner_alert=0
dist2=0
for index,row in df.iterrows():
    time=row["Timestamp"]
    pt = datetime.strptime(time,'%d-%m-%Y %H:%M')
    
    alt=getAltitude(row["Altitude"])
    dist=getDistance((float)(row["Band_Latitude"]),(float)(House.Latitude),(float)(row["Band_Longitude"]),(float)(House.Longitude))
    if(lat>0 and long>0):
        dist2=getDistance((float)(row["Band_Latitude"]),(float)(lat),(float)(row["Band_Longitude"]),(float)(long))
    lat=row["Band_Latitude"]
    long=row["Band_Longitude"]
    if(dist2>10000):
        print("\nTime: ",pt)
        print("Location Hardware has Failed")
        continue
    loc="Inside"
    if(dist>800):
        loc="Outside"
    user=User(loc)
    
    mobile=Mobile(row["Mobile_Alarm"],row["Mobile_Volume"],row["Mobile_Mic"],row["Mobile_Battery"],row["Latitude"],row["Longitude"],alt)

    alt=getAltitude(row["Band_Altitude"])
    band=FitBand(row["Heart_Rate"],row["SpO2"],row["Fitness_Battery"],row["Temperature"],row["Band_Latitude"],row["Band_Longitude"],alt)
    if(row["Heart_Rate"]<5):
        print("\nTime: ",pt)
        print("User is not wearing band")
        continue

    tv=TV(row["TV_Volume"],row["TV_Power"])
    if(row["TV_Power"]==0 and row["TV_Volume"]>0):
        print("\nTime: ",pt)
        print("TV Hardware has Failed")
        continue

    

    if (pt.time()>ls.time() and pt.time()<le.time() and lunch_alert==0):
        lunch_alert=1
        print("\nTime: ",pt)
        print("It's Lunch Time. Available nearby restaurants:")
        for i in range (len(Resto)):
            if(Resto[i].Status==1):
                rdist=getDistance((float)(Resto[i].Latitude),(float)(band.Latitude),(float)(Resto[i].Longitude),(float)(band.Longitude))
                if(rdist<1000):
                    print("")
                    print(Resto[i].Name)
                    print("Recommended Dishes:")
                    for j in range(len(Resto[i].History)):
                        if Resto[i].History[j][0] in Resto[i].Available:
                            print(Resto[i].History[j][0])

    elif (pt.time()>ds.time() and pt.time()<de.time() and dinner_alert==0):
        dinner_alert=1
        print("\nTime: ",pt)
        print("It's Dinner Time. Available nearby restaurants:")
        for i in range (len(Resto)):
            if(Resto[i].Status==1):
                rdist=getDistance((float)(Resto[i].Latitude),(float)(band.Latitude),(float)(Resto[i].Longitude),(float)(band.Longitude))
                if(rdist<1000):
                    print("")
                    print(Resto[i].Name)
                    print("Recommended Dishes:")
                    for j in range(len(Resto[i].History)):
                        if Resto[i].History[j][0] in Resto[i].Available:
                            print(Resto[i].History[j][0])

    if (pt.time()>le.time() and lunch_alert==1):
        lunch_alert=0
    
    if(pt.time()>de.time() and dinner_alert==1):
        dinner_alert=0

    if(user.Status=="Inside"):
        if(tv.Status==1 and mobile.Mic==1 and tv.Volume>0):
            tv.Volume=0
            print("\nTime: ",pt)
            print("User is speaking on phone. We reduce TV Volume.")
        elif(tv.Status==1 and tv.Volume==0 and mobile.Mic==0):
            tv.Volume=5
            print("\nTime: ",pt)
            print("User has finished his call. We increase TV Volume.")
    else:
        if(tv.Status==1):
            tv.Status=0
            print("\nTime: ",pt)
            print("User is outside. So we switch off the TV.")

    if(band.Heart_Rate>100 and band.oxygen<90):
        print("\nTime: ",pt)
        print("ALERT! USER IS FACING HEART ISSUES. CALLING AMBULANCE!")

    elif(band.oxygen<80):
        print("\nTime: ",pt)
        print("ALERT! USER FACING BREATHING ISSUES. CALLING AMBULANCE!")

    if(band.Temp>100):
        print("\nTime: ",pt)
        print("ALERT! USER HAS FEVER. CALLING AMBULANCE!")

    elif(band.Temp<80 and band.Temp>40):
        print("\nTime: ",pt)
        print("ALERT! USER HAS HYPOTHERMIA. CALLING AMBULANCE!")
    elif(band.Temp<-480):
        print("\nTime: ",pt)
        print("ALERT! USER HAS A WRONG READING.")

    if(mobile.Alarm==1 and Alarm_status==0):
        Alarm_status=1
        print("\nTime: ",pt)
        print("Alarm started ringing.")

    elif(mobile.Alarm==1 and Alarm_status==1):
        if(mobile.Volume<10):
            print("\nTime: ",pt)
            print("User didn't wake up. We increase volume.")
            mobile.Volume=mobile.Volume+1
        else:
            print("\nTime: ",pt)
            print("User didn't wake up. Max volume reached.")

    if(Alarm_status==1 and mobile.Alarm==0):
        print("\nTime: ",pt)
        print("User Woke up.")
        Alarm_status=0
        mobile.Volume=5


Time:  2021-10-12 06:30:00
Alarm started ringing.

Time:  2021-10-12 06:35:00
User didn't wake up. We increase volume.

Time:  2021-10-12 06:40:00
User didn't wake up. We increase volume.

Time:  2021-10-12 06:45:00
User Woke up.

Time:  2021-10-12 09:00:00
User is speaking on phone. We reduce TV Volume.

Time:  2021-10-12 09:25:00
User has finished his call. We increase TV Volume.

Time:  2021-10-12 11:55:00
Location Hardware has Failed

Time:  2021-10-12 13:25:00
It's Lunch Time. Available nearby restaurants:

BEDROCK
Recommended Dishes:
Burger
Thali
Pizza

BERTHA'S RESTAURANT
Recommended Dishes:
Thali
Pizza

BIRCHES RESTAURANT
Recommended Dishes:
Thali
Pizza

BLARNEY STONE INN
Recommended Dishes:
Thali
Pizza

AMBROSIA'S
Recommended Dishes:
Burger
Thali
Pizza

Time:  2021-10-12 17:39:00
Location Hardware has Failed

Time:  2021-10-12 20:04:00
It's Dinner Time. Available nearby restaurants:

BAY CAFE
Recommended Dishes:
Thali
Pizza

BELT STREET PUB
Recommended Dishes:
Thali
Pizza

BE

# Optimization:

1) Our system doesn't have any non deterministic activation fault as the rules are determined and if there is any place where it might get stuck, we have used the continue keyword to skip that portion.
2) We have 3 dead contexts: We take the mobile volume, mobile battery and fitness band battery but don't use it for any rules becoming waste of space.
3) We have 1 dead decision fault: We have a case where we check if temperature is below 0K which is not possible at all.
4) We do not have adaptation race fault as we update the table after running all context behaviours one after the other and hence escaping the infinite loop.
5) We do not have adaptation cycle fault as well as we have flags for the conditions where this might occur such as recommending restaurants.

# Case Based Reasoning

### 1) If Owner not in home and TV is on then switch off TV.<br> New Case: If we cannot detect if Owner is home or not, we check owner's previous saved location and work with that.

In [244]:
cols=df.columns
lat=-1
long=-1
for index,row in df.iterrows():
    time=row["Timestamp"]
    pt = datetime.strptime(time,'%d-%m-%Y %H:%M')
    alt=getAltitude(row["Altitude"])
    if(np.isnan(row["Band_Latitude"]) or np.isnan(row["Band_Longitude"])):
        row["Band_Latitude"]=lat
        row["Band_Longitude"]=long
        print("\nTime: ",pt)
        print("Cannot access location. Using previous data.")
    dist=getDistance((float)(row["Band_Latitude"]),(float)(House.Latitude),(float)(row["Band_Longitude"]),(float)(House.Longitude))
    lat=row["Band_Latitude"]
    long=row["Band_Longitude"]
    loc="Inside"
    if(dist>800):
        loc="Outside"
    user=User(loc)
    
    mobile=Mobile(row["Mobile_Alarm"],row["Mobile_Volume"],row["Mobile_Mic"],row["Mobile_Battery"],row["Latitude"],row["Longitude"],alt)

    alt=getAltitude(row["Band_Altitude"])
    band=FitBand(row["Heart_Rate"],row["SpO2"],row["Fitness_Battery"],row["Temperature"],row["Band_Latitude"],row["Band_Longitude"],alt)
    tv=TV(row["TV_Volume"],row["TV_Power"])

    if(user.Status=="Outside"):
        if(tv.Status==1):
            tv.Status=0
            print("\nTime: ",pt)
            print("User is outside. So we switch off the TV.")
    


Time:  2021-10-12 11:55:00
User is outside. So we switch off the TV.

Time:  2021-10-12 12:00:00
Cannot access location. Using previous data.

Time:  2021-10-12 12:00:00
User is outside. So we switch off the TV.

Time:  2021-10-12 15:15:00
Cannot access location. Using previous data.

Time:  2021-10-13 11:54:00
User is outside. So we switch off the TV.


### 2) If Owner not in home and time to eat we give Restaurants and Dish recommendation based on past orders.<br> New Case: If we do not have past order history for the given resto, we give the recoms based on other restaurants.

In [245]:
cols=df.columns
lunch_start="13:20"
lunch_end="14:00"
dinner_start="20:00"
dinner_end="21:00"
ls = datetime.strptime(lunch_start,'%H:%M')
le = datetime.strptime(lunch_end,'%H:%M')
ds = datetime.strptime(dinner_start,'%H:%M')
de = datetime.strptime(dinner_end,'%H:%M')
lat=-1
long=-1
Alarm_status=0
lunch_alert=0
dinner_alert=0
dist2=0
for index,row in df.iterrows():
    time=row["Timestamp"]
    pt = datetime.strptime(time,'%d-%m-%Y %H:%M')
    
    alt=getAltitude(row["Altitude"])
    dist=getDistance((float)(row["Band_Latitude"]),(float)(House.Latitude),(float)(row["Band_Longitude"]),(float)(House.Longitude))
    loc="Inside"
    if(dist>800):
        loc="Outside"
    user=User(loc)
    
    mobile=Mobile(row["Mobile_Alarm"],row["Mobile_Volume"],row["Mobile_Mic"],row["Mobile_Battery"],row["Latitude"],row["Longitude"],alt)

    alt=getAltitude(row["Band_Altitude"])
    band=FitBand(row["Heart_Rate"],row["SpO2"],row["Fitness_Battery"],row["Temperature"],row["Band_Latitude"],row["Band_Longitude"],alt)

    if (pt.time()>ls.time() and pt.time()<le.time() and lunch_alert==0):
        lunch_alert=1
        print("\nTime: ",pt)
        print("It's Lunch Time. Available nearby restaurants:")
        for i in range (len(Resto)):
            if(Resto[i].Status==1):
                rdist=getDistance((float)(Resto[i].Latitude),(float)(band.Latitude),(float)(Resto[i].Longitude),(float)(band.Longitude))
                if(rdist<1000):
                    print("")
                    print(Resto[i].Name)
                    print("Recommended Dishes:")
                    if(len(Resto[i].History)>0):
                        for j in range(len(Resto[i].History)):
                            if Resto[i].History[j][0] in Resto[i].Available:
                                print(Resto[i].History[j][0])
                    else:
                        print("RESTO HISTORY UNAVAILABLE. CHECKING GLOBAL HISTORY.")
                        for j in range(len(histall)):
                            if histall[j][0] in Resto[i].Available:
                                print(histall[j][0])

    elif (pt.time()>ds.time() and pt.time()<de.time() and dinner_alert==0):
        dinner_alert=1
        print("\nTime: ",pt)
        print("It's Dinner Time. Available nearby restaurants:")
        for i in range (len(Resto)):
            if(Resto[i].Status==1):
                rdist=getDistance((float)(Resto[i].Latitude),(float)(band.Latitude),(float)(Resto[i].Longitude),(float)(band.Longitude))
                if(rdist<1000):
                    print("")
                    print(Resto[i].Name)
                    print("Recommended Dishes:")
                    if(len(Resto[i].History)>0):
                        for j in range(len(Resto[i].History)):
                            if Resto[i].History[j][0] in Resto[i].Available:
                                print(Resto[i].History[j][0])
                    else:
                        print("RESTO HISTORY UNAVAILABLE. CHECKING GLOBAL HISTORY.")
                        for j in range(len(histall)):
                            if histall[j][0] in Resto[i].Available:
                                print(histall[j][0])

    if (pt.time()>le.time() and lunch_alert==1):
        lunch_alert=0
    
    if(pt.time()>de.time() and dinner_alert==1):
        dinner_alert=0



Time:  2021-10-12 13:25:00
It's Lunch Time. Available nearby restaurants:

BEDROCK
Recommended Dishes:
Burger
Thali
Pizza

BERTHA'S RESTAURANT
Recommended Dishes:
Thali
Pizza

BIRCHES RESTAURANT
Recommended Dishes:
Thali
Pizza

BLARNEY STONE INN
Recommended Dishes:
Thali
Pizza

AMBROSIA'S
Recommended Dishes:
Burger
Thali
Pizza

Time:  2021-10-12 20:04:00
It's Dinner Time. Available nearby restaurants:

BAY CAFE
Recommended Dishes:
Thali
Pizza

BELT STREET PUB
Recommended Dishes:
Thali
Pizza

BELVEDERE TOWERS CAF
Recommended Dishes:
Thali
Pizza

BERT'S EATS & DRINKS (3103-05)
Recommended Dishes:
Thali
Pizza

BILL'S 60'S & 70'S CLUB
Recommended Dishes:
Rajma Chawal
Pizza
Dosa

BILL'S CAFE
Recommended Dishes:
Thali
Pizza

BIRDS OF A FEATHER
Recommended Dishes:
Thali
Pizza

ALLEN RESTAURANT
Recommended Dishes:
Thali
Pizza

ALOHA TOKYO
Recommended Dishes:
RESTO HISTORY UNAVAILABLE. CHECKING GLOBAL HISTORY.
Thali
Pizza
Dosa

Time:  2021-10-13 13:24:00
It's Lunch Time. Available nearby rest